# 01_task1_data_prep: Step 1 - Data Preparation

**Objective:** Prepare JSE market data and Kasi CCI sentiment data for backtesting a sentiment-driven Risk-On/Risk-Off strategy

**Boss's Direction (from Yannick):**
- Use all 9 JSE indices (some highly correlated, some not)
- Discard SWIX for 2016-2025 regression (full historical period)
- Alternative: Test with 2020+ data including SWIX

**Steps:**
1. Load all 9 JSE indices and calculate monthly returns
2. Drop SWIX (only available from Dec 2020)
3. Load Kasi CCI sentiment data  
4. Convert both datasets to monthly frequency
5. Merge on matching dates
6. Export clean dataset

## Step 0: Import Required Libraries

In [48]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set pandas display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [49]:
# Load JSE data
jse_path = '../data/raw/JSE - Dataset_2016_2025_1.xlsx'
jse_df = pd.read_excel(jse_path, sheet_name='JSE data')

print("JSE Data Shape:", jse_df.shape)
print("\nFirst 5 rows:")
print(jse_df.head())

print("\nColumns:")
print(jse_df.columns.tolist())

print("\nDate range:")
print(f"From {jse_df['Date'].min()} to {jse_df['Date'].max()}")

JSE Data Shape: (521, 10)

First 5 rows:
        Date  Top 40 TR  Healthcare Index    Telcos  Consumer Staples  \
0 2025-12-19  19965.115           7441.10  22400.18          21939.99   
1 2025-12-12  19689.550           7139.08  21918.42          21663.98   
2 2025-12-05  19506.160           7210.44  22117.92          22076.78   
3 2025-11-28  19208.680           7173.99  22047.90          22194.99   
4 2025-11-21  18986.170           6967.77  23282.93          22027.30   

   Consumer Discretionary      SWIX  Industrials  Financials  \
0                95783.73  47193.83     31883.84    20715.33   
1                95434.30  46549.61     31388.68    20122.97   
2               100513.90  46190.99     31458.89    20148.66   
3                98097.58  45533.91     31443.86    19449.85   
4                97253.55  44947.03     32442.30    19540.74   

   ALSI Total Return  
0        21697.71661  
1        21401.53000  
2        21236.65000  
3        20934.56000  
4        20664.73000

In [54]:
# Ensure Date is a column (reset index if needed from previous runs)
if jse_df.index.name == 'Date':
    jse_df = jse_df.reset_index()

# Set Date as index and sort
jse_df = jse_df.sort_values('Date').set_index('Date')

# Use ALL 9 indices (excluding SWIX for now)
all_columns = jse_df.columns.tolist()
print(f"All available indices ({len(all_columns)}):")
print(all_columns)

# Remove SWIX for now (only available from Dec 2020) - will test separately
index_columns = [col for col in all_columns if col != 'SWIX']
jse_indices = jse_df[index_columns].copy()

print(f"\nUsing {len(index_columns)} indices (SWIX excluded for 2016-2025 period):")
print(index_columns)
print("\nFirst 5 rows:")
print(jse_indices.head())

# Resample to month-end
jse_monthly = jse_indices.resample('ME').last()

print("\nMonthly Index Values (first 10 months):")
print(jse_monthly.head(10))

# Calculate monthly returns
jse_returns = jse_monthly.pct_change() * 100

print("\nMonthly Returns (%):")
print(jse_returns.head(10))
print(f"\nShape: {jse_returns.shape}")
print(f"Date range: {jse_returns.index.min()} to {jse_returns.index.max()}")

All available indices (9):
['Top 40 TR', 'Healthcare Index', 'Telcos', 'Consumer Staples', 'Consumer Discretionary', 'SWIX', 'Industrials', 'Financials', 'ALSI Total Return']

Using 8 indices (SWIX excluded for 2016-2025 period):
['Top 40 TR', 'Healthcare Index', 'Telcos', 'Consumer Staples', 'Consumer Discretionary', 'Industrials', 'Financials', 'ALSI Total Return']

First 5 rows:
            Top 40 TR  Healthcare Index    Telcos  Consumer Staples  \
Date                                                                  
2016-01-01   6150.441          14496.14  13970.13          13540.25   
2016-01-08   5808.791          13677.86  13181.57          13345.98   
2016-01-15   5655.574          13098.81  12536.09          13299.22   
2016-01-22   5769.420          13172.91  12937.10          13382.18   
2016-01-29   5919.755          13712.89  14387.93          13191.83   

            Consumer Discretionary  Industrials  Financials  ALSI Total Return  
Date                                

In [56]:
# Load Kasi CCI data
cci_path = '../data/raw/Kasi CCI - Data_2016_2025_1.xlsx'
cci_df = pd.read_excel(cci_path, sheet_name='Kasi CCI')

print("Kasi CCI Data Shape:", cci_df.shape)
print("\nFirst 10 rows:")
print(cci_df.head(10))

print("\nData types:")
print(cci_df.dtypes)

print("\nDate range:")
print(f"From {cci_df['Date'].min()} to {cci_df['Date'].max()}")

print("\nCCI Statistics:")
print(cci_df['CCI'].describe())

Kasi CCI Data Shape: (113, 2)

First 10 rows:
         Date   CCI
0  01/07/2016  14.0
1  01/08/2016  14.0
2  01/09/2016  12.0
3  01/10/2016   1.0
4  01/11/2016   7.0
5  01/12/2016   2.0
6  01/01/2017  22.0
7  01/02/2017  17.0
8  01/03/2017 -14.0
9  01/04/2017  -8.0

Data types:
Date        str
CCI     float64
dtype: object

Date range:
From 01/01/2017 to 01/12/2024

CCI Statistics:
count    113.000000
mean      -1.721239
std       26.578299
min      -87.000000
25%       -9.000000
50%       -2.000000
75%        4.000000
max       70.000000
Name: CCI, dtype: float64


In [57]:
# Convert Date column to datetime format
cci_df['Date'] = pd.to_datetime(cci_df['Date'])

print("After conversion:")
print(cci_df.dtypes)

print("\nFirst 5 rows:")
print(cci_df.head())

print("\nDate range after conversion:")
print(f"From {cci_df['Date'].min()} to {cci_df['Date'].max()}")

After conversion:
Date    datetime64[us]
CCI            float64
dtype: object

First 5 rows:
        Date   CCI
0 2016-01-07  14.0
1 2016-01-08  14.0
2 2016-01-09  12.0
3 2016-01-10   1.0
4 2016-01-11   7.0

Date range after conversion:
From 2016-01-07 00:00:00 to 2025-01-11 00:00:00


In [58]:
# Check which columns have missing data
print("Data Completeness Report:")
print("="*60)

missing_counts = jse_indices.isnull().sum()
missing_pct = (missing_counts / len(jse_indices)) * 100

for col in jse_indices.columns:
    print(f"\n{col}:")
    print(f"  Missing: {missing_counts[col]} out of {len(jse_indices)} ({missing_pct[col]:.1f}%)")
    if missing_counts[col] > 0:
        first_valid = jse_indices[col].first_valid_index()
        print(f"  First valid date: {first_valid}")

print("\n" + "="*60)
print(f"Ready to merge: {jse_returns.shape[0]} months of returns data")
print(f"Date range: {jse_returns.index.min().date()} to {jse_returns.index.max().date()}")

Data Completeness Report:

Top 40 TR:
  Missing: 0 out of 521 (0.0%)

Healthcare Index:
  Missing: 0 out of 521 (0.0%)

Telcos:
  Missing: 0 out of 521 (0.0%)

Consumer Staples:
  Missing: 0 out of 521 (0.0%)

Consumer Discretionary:
  Missing: 0 out of 521 (0.0%)

Industrials:
  Missing: 0 out of 521 (0.0%)

Financials:
  Missing: 0 out of 521 (0.0%)

ALSI Total Return:
  Missing: 0 out of 521 (0.0%)

Ready to merge: 120 months of returns data
Date range: 2016-01-31 to 2025-12-31


In [61]:
# Final dataset: JSE returns (8 indices, no SWIX, start from 2016-02-29)
# Filter out 2016-01-31 (has NaN values from first row pct_change)
cutoff_date = pd.Timestamp('2016-02-29')
jse_returns_clean = jse_returns[jse_returns.index >= cutoff_date].dropna()

print("Final JSE Returns Dataset:")
print(f"  Shape: {jse_returns_clean.shape[0]} months of data")
print(f"  Indices: {list(jse_returns_clean.columns)}")
print(f"  Date range: {jse_returns_clean.index.min().date()} to {jse_returns_clean.index.max().date()}")

print("\nData Summary:")
print(jse_returns_clean.describe())

print("\nData Integrity Check:")
print(f"  Missing values: {jse_returns_clean.isnull().sum().sum()}")

print("\nFirst 5 rows - ready for merging with CCI:")
print(jse_returns_clean.head())

Final JSE Returns Dataset:
  Shape: 119 months of data
  Indices: ['Top 40 TR', 'Healthcare Index', 'Telcos', 'Consumer Staples', 'Consumer Discretionary', 'Industrials', 'Financials', 'ALSI Total Return']
  Date range: 2016-02-29 to 2025-12-31

Data Summary:
        Top 40 TR  Healthcare Index      Telcos  Consumer Staples  \
count  119.000000        119.000000  119.000000        119.000000   
mean     1.127892         -0.302881    0.665720          0.549674   
std      4.533313          6.374654    7.623900          4.953417   
min    -13.886619        -26.769389  -28.810876        -16.307315   
25%     -1.510528         -3.806881   -4.423211         -2.177773   
50%      1.482710         -0.534498    1.251576          0.864910   
75%      3.849920          4.355935    5.384276          3.288153   
max     17.299836         15.388644   17.572308         21.905156   

       Consumer Discretionary  Industrials  Financials  ALSI Total Return  
count              119.000000   119.000000

In [65]:
## Step 5: Convert CCI to Monthly Frequency (Set Date as Index)

In [71]:
# Export to CSV and Excel
interim_dir = '../data/interim'
os.makedirs(interim_dir, exist_ok=True)

# Export to CSV
csv_path = os.path.join(interim_dir, 'backtest_data.csv')
backtest_data.to_csv(csv_path)
print(f"✅ Exported to CSV: {csv_path}")

# Export to Excel
excel_path = os.path.join(interim_dir, 'backtest_data.xlsx')
backtest_data.to_excel(excel_path, sheet_name='Data')
print(f"✅ Exported to Excel: {excel_path}")

print("\n" + "="*60)
print("STEP 1: DATA PREPARATION COMPLETE!")
print("="*60)
print(f"\nFinal dataset ready for backtesting:")
print(f"  • {backtest_data.shape[0]} months of data")
print(f"  • {backtest_data.shape[1]-1} JSE indices + 1 CCI column")
print(f"  • Period: {backtest_data.index.min().date()} to {backtest_data.index.max().date()}")
print(f"  • No missing values: {backtest_data.isnull().sum().sum() == 0}")
print(f"\nNext steps: Build sentiment signal → Design allocation strategy → Backtest")

✅ Exported to CSV: ../data/interim/backtest_data.csv
✅ Exported to Excel: ../data/interim/backtest_data.xlsx

STEP 1: DATA PREPARATION COMPLETE!

Final dataset ready for backtesting:
  • 119 months of data
  • 8 JSE indices + 1 CCI column
  • Period: 2016-02-29 to 2025-12-31
  • No missing values: True

Next steps: Build sentiment signal → Design allocation strategy → Backtest


## Step 7: Export Clean Dataset to data/interim/

In [73]:
# Debug: Check the indices
print("Debug Info:")
print(f"JSE returns index type: {type(jse_returns_clean.index)}")
print(f"JSE returns index[0]: {jse_returns_clean.index[0]}")
print(f"CCI monthly index type: {type(cci_monthly.index)}")
print(f"CCI monthly index[0]: {cci_monthly.index[0]}")

# Merge JSE returns with CCI - use asof merge to find nearest dates
backtest_data = pd.merge_asof(
    jse_returns_clean.reset_index().sort_values('Date'),
    cci_monthly.reset_index().sort_values('Date'),
    on='Date',
    direction='nearest'
).set_index('Date').sort_index()

print("\nMerged Dataset:")
print(f"  Shape: {backtest_data.shape[0]} months × {backtest_data.shape[1]} columns")
print(f"  Columns: {list(backtest_data.columns)}")
print(f"  Date range: {backtest_data.index.min().date()} to {backtest_data.index.max().date()}")

print("\nFirst 5 rows:")
print(backtest_data.head())

print("\nData Integrity Check:")
print(f"  Missing values:\n{backtest_data.isnull().sum()}")

Debug Info:
JSE returns index type: <class 'pandas.DatetimeIndex'>
JSE returns index[0]: 2016-02-29 00:00:00
CCI monthly index type: <class 'pandas.DatetimeIndex'>
CCI monthly index[0]: 2016-01-07 00:00:00

Merged Dataset:
  Shape: 119 months × 9 columns
  Columns: ['Top 40 TR', 'Healthcare Index', 'Telcos', 'Consumer Staples', 'Consumer Discretionary', 'Industrials', 'Financials', 'ALSI Total Return', 'CCI']
  Date range: 2016-02-29 to 2025-12-31

First 5 rows:
            Top 40 TR  Healthcare Index     Telcos  Consumer Staples  \
Date                                                                   
2016-02-29  -0.471455          0.230367  -4.990919         -0.555268   
2016-03-31   6.437999          6.631171   3.504284          2.268919   
2016-04-30   0.635799          3.722801  14.030166         -4.101525   
2016-05-31   3.472426         -1.104561 -11.826468          6.908385   
2016-06-30  -5.156580          2.362974   9.363555         -8.097107   

            Consumer Discret

## Step 6: Merge JSE Returns with CCI Monthly Data

In [62]:
# Set Date as index for CCI data and resample to month-end
cci_monthly = cci_df.set_index('Date').sort_index()

# Resample to month-end (take last CCI reading of each month)
cci_monthly = cci_monthly.resample('ME').last()

print("CCI Data Resampled to Month-End:")
print(f"  Shape: {cci_monthly.shape}")
print(f"  Date range: {cci_monthly.index.min().date()} to {cci_monthly.index.max().date()}")

print("\nFirst 5 rows:")
print(cci_monthly.head())

print("\nLast 5 rows:")
print(cci_monthly.tail())

CCI Data with Date as Index:
  Shape: (113, 1)
  Date range: 2016-01-07 to 2025-01-11

First 5 rows:
             CCI
Date            
2016-01-07  14.0
2016-01-08  14.0
2016-01-09  12.0
2016-01-10   1.0
2016-01-11   7.0

Last 5 rows:
             CCI
Date            
2025-01-07  -9.0
2025-01-08  -8.0
2025-01-09  -2.0
2025-01-10  -7.0
2025-01-11 -17.0


## Step 4: Prepare Final JSE Dataset

## Step 3: Check Data Completeness